# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

### Ensure that the dependencies and the Geoapify API key is imported correctly

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
import sys

sys.path.append('../')
from config import geoapify_key

### From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [21]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database.csv")
city_data_df.head()

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Labuhan,ID,-6.8844,112.2051,82.02,76,100,5.08,overcast clouds
1,1,Avarua,CK,-21.2078,-159.7750,80.65,69,20,6.91,light rain
2,2,Erzurum,TR,40.0000,41.5000,22.68,85,100,2.80,overcast clouds
3,3,Half Moon Bay,US,37.4636,-122.4286,52.25,81,75,18.41,haze
4,4,Kanor,IN,24.4333,74.2667,37.76,37,0,7.81,clear sky


### Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [22]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


### Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [23]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]

# Display sample data
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Labuhan,ID,-6.8844,112.2051,82.02,76,100,5.08,overcast clouds
1,1,Avarua,CK,-21.2078,-159.7750,80.65,69,20,6.91,light rain
12,12,Atuona,PF,-9.8000,-139.0333,78.19,76,15,16.08,few clouds
16,16,Rikitea,PF,-23.1203,-134.9692,77.79,68,2,5.79,clear sky
18,18,Hithadhoo,MV,-0.6000,73.0833,79.36,67,70,4.32,broken clouds
...,...,...,...,...,...,...,...,...,...,...
551,551,Nabire,ID,-3.3667,135.4833,86.54,65,78,6.26,broken clouds
560,560,Toamasina,MG,-18.1667,49.3833,77.00,88,75,3.44,light rain
562,562,Kapoeta,SS,4.7717,33.5903,75.63,40,100,2.17,overcast clouds
564,564,Luganville,VU,-15.5333,167.1667,83.68,77,90,1.79,overcast clouds


### Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [24]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities_df = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Labuhan,ID,-6.8844,112.2051,82.02,76,100,5.08,overcast clouds
1,1,Avarua,CK,-21.2078,-159.7750,80.65,69,20,6.91,light rain
12,12,Atuona,PF,-9.8000,-139.0333,78.19,76,15,16.08,few clouds
16,16,Rikitea,PF,-23.1203,-134.9692,77.79,68,2,5.79,clear sky
18,18,Hithadhoo,MV,-0.6000,73.0833,79.36,67,70,4.32,broken clouds


### Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [26]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng
0,Labuhan,ID,82.02,overcast clouds,-6.8844,112.2051
1,Avarua,CK,80.65,light rain,-21.2078,-159.7750
12,Atuona,PF,78.19,few clouds,-9.8000,-139.0333
16,Rikitea,PF,77.79,clear sky,-23.1203,-134.9692
18,Hithadhoo,MV,79.36,broken clouds,-0.6000,73.0833
34,Georgetown,MY,84.13,few clouds,5.4112,100.3354
37,Bubaque,GW,77.27,broken clouds,11.2833,-15.8333
40,Daru,PG,82.04,overcast clouds,-9.0763,143.2092
49,Meulaboh,ID,76.12,overcast clouds,4.1363,96.1285
51,Kapaa,US,81.12,clear sky,22.0752,-159.3190


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [27]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Labuhan,ID,82.02,overcast clouds,-6.8844,112.2051,
1,Avarua,CK,80.65,light rain,-21.2078,-159.7750,
12,Atuona,PF,78.19,few clouds,-9.8000,-139.0333,
16,Rikitea,PF,77.79,clear sky,-23.1203,-134.9692,
18,Hithadhoo,MV,79.36,broken clouds,-0.6000,73.0833,
34,Georgetown,MY,84.13,few clouds,5.4112,100.3354,
37,Bubaque,GW,77.27,broken clouds,11.2833,-15.8333,
40,Daru,PG,82.04,overcast clouds,-9.0763,143.2092,
49,Meulaboh,ID,76.12,overcast clouds,4.1363,96.1285,
51,Kapaa,US,81.12,clear sky,22.0752,-159.3190,


### Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [28]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

### Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [29]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Set initial parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Labuhan - nearest hotel: No hotel found
Avarua - nearest hotel: Paradise Inn
Atuona - nearest hotel: Pearl Resort
Rikitea - nearest hotel: Chez Bianca & Benoit
Hithadhoo - nearest hotel: Pebbles Inn
Georgetown - nearest hotel: Page 63 hostel
Bubaque - nearest hotel: Chez Julio
Daru - nearest hotel: Kuki
Meulaboh - nearest hotel: Hotel Meuligou
Kapaa - nearest hotel: Pono Kai Resort
Hobart - nearest hotel: Vibe Hotel Hobart
Kavaratti - nearest hotel: No hotel found
Hilo - nearest hotel: Dolphin Bay Hotel
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Victoria Point - nearest hotel: Seminara Appartments
Faanui - nearest hotel: Oa Oa Lodge
Vaini - nearest hotel: Keleti International Resort
Belawan - nearest hotel: Danau Toba International Cottage Belawan
Santa Cruz Cabralia - nearest hotel: Campo Bahia Resort
Mogadishu - nearest hotel: SYL Hotel
The Valley - nearest hotel: CeBlue
Presidencia Roque Saenz Pena - nearest hotel: Hotel Presidente
Loren

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Labuhan,ID,82.02,overcast clouds,-6.8844,112.2051,No hotel found
1,Avarua,CK,80.65,light rain,-21.2078,-159.7750,Paradise Inn
12,Atuona,PF,78.19,few clouds,-9.8000,-139.0333,Pearl Resort
16,Rikitea,PF,77.79,clear sky,-23.1203,-134.9692,Chez Bianca & Benoit
18,Hithadhoo,MV,79.36,broken clouds,-0.6000,73.0833,Pebbles Inn
...,...,...,...,...,...,...,...
551,Nabire,ID,86.54,broken clouds,-3.3667,135.4833,Hotel Anggrek
560,Toamasina,MG,77.00,light rain,-18.1667,49.3833,Androranga
562,Kapoeta,SS,75.63,overcast clouds,4.7717,33.5903,Kuleu
564,Luganville,VU,83.68,overcast clouds,-15.5333,167.1667,Beachfront Resort


### Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [30]:
len(hotel_df)

134

In [31]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Display sample data
clean_hotel_df


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Avarua,CK,80.65,light rain,-21.2078,-159.7750,Paradise Inn
12,Atuona,PF,78.19,few clouds,-9.8000,-139.0333,Pearl Resort
16,Rikitea,PF,77.79,clear sky,-23.1203,-134.9692,Chez Bianca & Benoit
18,Hithadhoo,MV,79.36,broken clouds,-0.6000,73.0833,Pebbles Inn
34,Georgetown,MY,84.13,few clouds,5.4112,100.3354,Page 63 hostel
...,...,...,...,...,...,...,...
551,Nabire,ID,86.54,broken clouds,-3.3667,135.4833,Hotel Anggrek
560,Toamasina,MG,77.00,light rain,-18.1667,49.3833,Androranga
562,Kapoeta,SS,75.63,overcast clouds,4.7717,33.5903,Kuleu
564,Luganville,VU,83.68,overcast clouds,-15.5333,167.1667,Beachfront Resort


In [32]:
len(clean_hotel_df)

97

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [33]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [34]:
# Configure the map plot
hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Country", "Hotel Name",  "Weather Description"]
    )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Hotel Name,Weather Description)